In [1]:
#Import basic necessary datasets
import pandas as pd
import numpy as np

In [18]:
#Read the datasets and view the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [19]:
#filling null values
train.bmi.fillna(value=0, axis=0, inplace=True)
test.bmi.fillna(value=0, axis=0, inplace=True)

train.smoking_status.fillna(value='no information', axis=0, inplace=True)
test.smoking_status.fillna(value='no information', axis=0, inplace=True)

In [20]:
#grouping the data
train.age = np.digitize(train.age, bins=np.arange(start=train.age.min(),stop=train.age.max(),step=10))
test.age = np.digitize(test.age, bins=np.arange(start=test.age.min(),stop=test.age.max(),step=10))

train.avg_glucose_level = np.digitize(train.avg_glucose_level, bins=np.arange(start=train.avg_glucose_level.min(),stop=train.avg_glucose_level.max(),step=50))
test.avg_glucose_level = np.digitize(test.avg_glucose_level, bins=np.arange(start=test.avg_glucose_level.min(),stop=test.avg_glucose_level.max(),step=50))

train.bmi = np.digitize(train.bmi, bins=np.arange(start=train.bmi.min(),stop=train.bmi.max(),step=10))
test.bmi = np.digitize(test.bmi, bins=np.arange(start=test.bmi.min(),stop=test.bmi.max(),step=10))

In [21]:
train.bmi[train.bmi.isin([7,8,9,10])] = 6
test.bmi[test.bmi.isin([7,8,9,10])] = 6

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train.gender = le.fit_transform(train.gender.values)
train.ever_married = le.fit_transform(train.ever_married.values)
train.work_type = le.fit_transform(train.work_type.values)
train.Residence_type = le.fit_transform(train.Residence_type.values)
train.smoking_status = le.fit_transform(train.smoking_status.values)
train.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,1,1,0,0,0,4,0,1,2,2,0
1,30468,1,6,1,0,1,2,1,1,4,1,0
2,16523,0,1,0,0,0,2,1,2,2,2,0
3,56543,0,7,0,0,1,2,0,1,4,0,0
4,46136,1,2,0,0,0,1,0,3,2,2,0


In [23]:
test.gender = le.fit_transform(test.gender.values)
test.ever_married = le.fit_transform(test.ever_married.values)
test.work_type = le.fit_transform(test.work_type.values)
test.Residence_type = le.fit_transform(test.Residence_type.values)
test.smoking_status = le.fit_transform(test.smoking_status.values)
test.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,1,8,0,0,1,2,1,1,3,0
1,61829,0,8,0,1,1,3,0,3,3,0
2,14152,0,2,0,0,0,4,0,1,3,2
3,12997,1,3,0,0,0,2,1,1,3,2
4,40801,0,7,0,0,1,0,0,1,3,1


In [24]:
#drop id column
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [9]:
#get dummy variables
train = pd.get_dummies(train, columns=['gender','age','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'],drop_first=True)
test = pd.get_dummies(test, columns=['gender','age','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'],drop_first=True)

In [25]:
#Take independent features into x and depenent feature into y
X = train.drop('stroke', axis=1)
y = train.stroke

In [26]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_scaled = std.fit_transform(X)
X_scaled

array([[ 1.20108119, -1.64838568, -0.32129564, ..., -0.57595987,
        -1.39687604,  0.59661818],
       [ 1.20108119,  0.58894336,  3.11239826, ..., -0.57595987,
         0.78908751, -0.4612414 ],
       [-0.83084068, -1.64838568, -0.32129564, ...,  0.56643117,
        -1.39687604,  0.59661818],
       ..., 
       [-0.83084068,  1.93134079,  3.11239826, ..., -0.57595987,
        -0.30389427, -1.51910099],
       [ 1.20108119, -0.30598826, -0.32129564, ..., -0.57595987,
         0.78908751, -0.4612414 ],
       [-0.83084068,  1.93134079, -0.32129564, ..., -0.57595987,
        -0.30389427, -0.4612414 ]])

In [27]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state=123)

In [13]:
#import keras libraries
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras.optimizers import Adam, rmsprop

Using TensorFlow backend.


In [28]:
#build a classifier function
def build_classifier():
    classifier = Sequential()  # initiate the model
    classifier.add(Dense(200, input_shape=(10,), activation='relu')) # input layer
    classifier.add(Dense(150, activation='relu'))                  # hidden layer
    classifier.add(Dense(100, activation='relu'))                  # hidden layer
    classifier.add(Dense(50, activation='relu'))                  # hidden layer
    #classifier.add(Dropout(0.25))
    classifier.add(Dense(1, activation = 'sigmoid')) #output layer
    classifier.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    return classifier

In [29]:
#create a KerasClassifier object and call build_classifier function
classifier = KerasClassifier(build_fn=build_classifier, batch_size=1000, nb_epoch=20, verbose=2)

In [16]:
from IPython.core.interactiveshell import InteractiveShell #To print multiple outputs
InteractiveShell.ast_node_interactivity = 'all'

In [30]:
accuracies = cross_val_score(classifier, X_train, y_train, cv=10)

Epoch 1/10
 - 1s - loss: 0.2120 - acc: 0.9764
Epoch 2/10
 - 0s - loss: 0.0825 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.0772 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.0753 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0741 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0732 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0727 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0722 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0723 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0719 - acc: 0.9821
Epoch 1/10
 - 1s - loss: 0.2594 - acc: 0.9322
Epoch 2/10
 - 0s - loss: 0.0856 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.0778 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.0758 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0745 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0737 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0731 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0725 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0721 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0716 - acc: 0.9821
Epoch 1/10
 - 1s - loss: 0.1982 - acc: 0.9768
Epoch 2/10
 - 0s - loss: 0.0823 

In [31]:
accuracies.mean()
accuracies.std()

0.98211405593129353

0.0012113863358717109

In [32]:
classifier.fit(X_train, y_train)

Epoch 1/10
 - 1s - loss: 0.1765 - acc: 0.9818
Epoch 2/10
 - 0s - loss: 0.0801 - acc: 0.9821
Epoch 3/10
 - 0s - loss: 0.0762 - acc: 0.9821
Epoch 4/10
 - 0s - loss: 0.0745 - acc: 0.9821
Epoch 5/10
 - 0s - loss: 0.0736 - acc: 0.9821
Epoch 6/10
 - 0s - loss: 0.0731 - acc: 0.9821
Epoch 7/10
 - 0s - loss: 0.0724 - acc: 0.9821
Epoch 8/10
 - 0s - loss: 0.0720 - acc: 0.9821
Epoch 9/10
 - 0s - loss: 0.0715 - acc: 0.9821
Epoch 10/10
 - 0s - loss: 0.0716 - acc: 0.9821


In [33]:
from sklearn import metrics
y_pred = classifier.predict_proba(X_test)
metrics.roc_auc_score(y_test,y_pred[:,1])

0.86548057997733197

In [34]:
test_pred = classifier.predict_proba(np.array(test))
test_pred.max()

0.99767798

In [35]:
sample['stroke'] = test_pred[:,1]
sample.to_csv('sample_submission.csv',index=False)
sample.head()

,id,stroke
0,36306,0.012264
1,61829,0.023151
2,14152,0.019868
3,12997,0.029502
4,40801,0.015789


In [36]:
sample.stroke.max()

0.18072312